# Working with APIs in Python

You may have heard of an "API", and you may even have a notion that APIs might be a tool for getting access to interesting information, but they're difficult to use without any coding. With a Jupyter notebook, you still need to write code, but you don't need to write one perfect script to do exactly what you want the first time around. You can experiment and look at what each line of code does.

As an example, the Harvard Art Museums provide access to a ton of information about their collections, available through a pretty simple API.

Here's some [API documentation](https://github.com/harvardartmuseums/api-docs/blob/master/sections/object.md) and a [sample API query](https://api.harvardartmuseums.org/object?q=totalpageviews:0&size=10&apikey=6dd7b080-3635-11ea-851d-67c3f7f3cf76).

The API documentation kind of looks like documentation for an advanced search feature if you squint a little, but I bet the sample query doesn't look very nice to you. It is, however, very easy to work with in a programming language like Python. A Jupyter notebook is a great place to explore an API, since you can execute individual cells of code, look at the results, and then code some more. It's much more interactive and exploratory than just writing a script, but there's more scaffolding than you'd find just executing code in your terminal.

In this notebook, I've set up a little toy script using the Harvard Art Museums API that allows you to search the collections by color. It turns out you have to specify the color in hexadecimal notation (something like "#fae1e1"), and you can't just put in an arbitrary color; only certain colors will turn up anything. I've put together a list of colors that will work and a function for searching. All you have to do is copy a color from the list and put it between the quotes in the function below, and you'll get a sampling of ten works of art that feature that color.

In [ ]:
import requests
import random
import csv
from IPython.core.display import display, HTML

APIKEY = "6dd7b080-3635-11ea-851d-67c3f7f3cf76"

colors = []
with open("colors.csv", "r") as fp:
    reader = csv.reader(fp)
    for row in reader:
        colors.append(row[0])


def color_search(color):
    color = str(color)
    if color[0] != "#":
        color = "#" + color
    Q = {
        "apikey": APIKEY,
        "color": color,
        "size": 10,
        "sort": f"random:{random.randint(0,9999999)}"
    }
    R = requests.get("https://api.harvardartmuseums.org/object", params=Q)
    info  = R.json()['info']
    records = R.json()['records']
    
    for entry in records:
        if "colors" in entry:
            for c in entry['colors']:
                if c['color'] not in colors:
                    colors.append(c['color'])
                    print(f"new color found: {c['color']}")
    
    output = f"<p>Search returned {info['totalrecords']} results. Showing first {info['totalrecordsperquery']}.</p><table>"
    for r in records:
        row = f'<tr><td><a href="{r["url"]}" target="_blank">{r["title"]}</td><td style="width:300px;">'
        for color in r['colors']:
            row += f'<div style="background-color:{color["color"]}; width:{color["percent"] * 100}%; float:left;">&nbsp;</div>'
        row += "</td>"
        try:
            imgUrl = r["images"][0]["iiifbaseuri"].replace("iiif","view")
            row += f'<td><a href="{imgUrl}" target="_blank"><img style="height:100px;" src="{imgUrl}"></img></a></td>'
        except IndexError:
            row += "<td></td>"
        row += "</tr>"
        output += row
    output += "</table>"
    display(HTML(output))
    return 0

In [ ]:
random.shuffle(colors)
htmlOutput = '<table style="-webkit-text-stroke: 1.5px #000000; color:#ffffff; font-size:26px; font-family:Impact;"><tr>'
counter = 0
num_columns = 6
for c in colors:
    htmlOutput += f'<td style="background-color:{c}; padding:25px;">{c}</td>'
    if counter >= num_columns - 1:
        htmlOutput += "</tr><tr>"
        counter = 0
    else:
        counter += 1
htmlOutput += "</tr></table>"
display(HTML(htmlOutput))

In [ ]:
color_search("323264")